# Notes on candlestick tradding patterns

In [1]:
import MetaTrader5 as mt5
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

from IPython.display import display # to display pandas dataframe

In [3]:
# connect to MetaTrader5 Terminal
isConnected = mt5.initialize()

if not isConnected:
    print("initialize() failed, error code =", mt5.last_error())
else: print("initialize() success") 




initialize() success


In [4]:
# this function retrieves olhc data from MT5 account and return a dataframe
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s') # convert timestamp to datetime
    
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]

In [18]:
symbol = 'XAUUSDm'
timeframe = mt5.TIMEFRAME_M5
start_datetime = datetime(2023, 11, 1)
end_datetime = datetime(2023, 11, 5)

ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
ohlc_df

,time,open,high,low,close
0,2023-11-01 04:00:00,1977.079,1977.864,1976.838,1977.767
1,2023-11-01 04:05:00,1977.771,1977.827,1977.200,1977.418
2,2023-11-01 04:10:00,1977.440,1977.662,1975.758,1975.814
3,2023-11-01 04:15:00,1975.870,1976.219,1975.231,1975.867
4,2023-11-01 04:20:00,1975.864,1976.382,1975.582,1975.731
...,...,...,...,...,...
751,2023-11-03 20:35:00,1992.453,1992.698,1992.413,1992.504
752,2023-11-03 20:40:00,1992.504,1992.506,1992.307,1992.451
753,2023-11-03 20:45:00,1992.414,1992.417,1992.212,1992.280
754,2023-11-03 20:50:00,1992.291,1992.580,1992.227,1992.471


In [19]:
# visualizing the ohlc data
fig = go.Figure(data=[go.Candlestick(x=ohlc_df['time'],
                open=ohlc_df['open'],
                high=ohlc_df['high'],
                low=ohlc_df['low'],
                close=ohlc_df['close'])# (x, open, high, low, close) for candlestick
                      ]
                )

fig.update_layout(height=600, xaxis_rangeslider_visible=False)
fig.show()

---

### hammer patterns

A hammer candlestick pattern is a bullish reversal pattern typically formed after a downtrend, indicating a potential price reversal to the upside. It features a small real body near the top of the candlestick and a long lower shadow at least twice the size of the real body, with little to no upper shadow. 

The long lower wick suggests that sellers pushed the price significantly lower during the trading session, but strong buying pressure drove the price back up near the opening level by the close. This price action signals that buyers may be gaining control.

 The color of the hammer's real body (green or red) can vary, but a green (or white) body is generally considered more bullish. When confirmed by subsequent bullish price movement, the hammer pattern is a reliable indicator of a potential trend reversal.

<div style="text-align: center;">
<img src=".\images\What-is-a-Hammer-Candlestick-1024x640.png" alt="Hammer Pattern Image" width="400" height="300">
</div>

In [20]:
# finding hammer patterns
def find_hammer_pattern(candle):
    candle_range = candle['high'] - candle['low']
    
    # logic for bullish hammer patterns
    bull_cond1 = candle['close'] > candle['open']
    # check if the hammer pattern is contained in the upper 30% of the candle
    bull_cond2 = (candle['high'] - candle['open']) / candle_range <= 0.3 
    
    
    # logic for bearish hammer patterns
    bear_cond1 = candle['close'] < candle['open']
    # check if the hammer pattern is contained in the lower 30% of the candle
    bear_cond2 = (candle['open'] - candle['low']) / candle_range <= 0.3 

    
    if bull_cond1 and bull_cond2:
        return True
    elif bear_cond1 and bear_cond2:
        return True
    else:
        return False

ohlc_df['hammer_pattern'] = ohlc_df.apply(find_hammer_pattern, axis=1)
ohlc_df

,time,open,high,low,close,hammer_pattern
0,2023-11-01 04:00:00,1977.079,1977.864,1976.838,1977.767,False
1,2023-11-01 04:05:00,1977.771,1977.827,1977.200,1977.418,False
2,2023-11-01 04:10:00,1977.440,1977.662,1975.758,1975.814,False
3,2023-11-01 04:15:00,1975.870,1976.219,1975.231,1975.867,False
4,2023-11-01 04:20:00,1975.864,1976.382,1975.582,1975.731,False
...,...,...,...,...,...,...
751,2023-11-03 20:35:00,1992.453,1992.698,1992.413,1992.504,False
752,2023-11-03 20:40:00,1992.504,1992.506,1992.307,1992.451,False
753,2023-11-03 20:45:00,1992.414,1992.417,1992.212,1992.280,False
754,2023-11-03 20:50:00,1992.291,1992.580,1992.227,1992.471,False


---
# engulfing pattern

An engulfing candlestick pattern is a powerful reversal pattern that occurs in financial markets and signals a potential shift in price direction. There are two types of engulfing patterns: bullish engulfing and bearish engulfing. A bullish engulfing pattern forms after a downtrend and consists of a small bearish candle (typically red) followed by a larger bullish candle (typically green) that completely "engulfs" the body of the previous candle. 

This indicates that buyers have taken control, suggesting a potential upward price reversal. Conversely, a bearish engulfing pattern appears after an uptrend and consists of a small bullish candle followed by a larger bearish candle that engulfs the previous candle’s body, signaling that sellers have gained dominance and a downward reversal may follow. 

Engulfing patterns are more significant when they appear after extended trends and when supported by high trading volume, as this increases the likelihood of a substantial price movement in the indicated direction. Traders often use engulfing patterns alongside other technical indicators to confirm the strength of the signal before taking action.

<center>
  <img src="./images/engulfing.jpeg" alt="Centered Image" width="400">
</center>



In [22]:
# finding engulfing patterns
ohlc_df['previous_open'] = ohlc_df['open'].shift(1)
ohlc_df['previous_high'] = ohlc_df['high'].shift(1)
ohlc_df['previous_low'] = ohlc_df['low'].shift(1)
ohlc_df['previous_close'] = ohlc_df['close'].shift(1)

def find_engulfing_pattern(candle):
    
    bull_cond1 = candle['close'] > candle['open'] # bull candle condition
    bull_cond2 = candle['close'] > candle['previous_high'] # engulfment condition
    bull_cond3 = candle['previous_open'] > candle['previous_close'] # previous candle must be a bear candle
    
    bear_cond1 = candle['close'] < candle['open'] # bear candle condition
    bear_cond2 = candle['close'] < candle['previous_low'] # engulfment condition
    bear_cond3 = candle['previous_open'] < candle['previous_close'] # previous candle must be a bull candle
    
    # special condition - engulfing candle body is 1.5 times as long as previous candle range
    special_cond = abs(candle['open'] - candle['close']) / (candle['previous_high'] - candle['previous_low']) >= 1.5
    
    if bull_cond1 and bull_cond2 and bull_cond3 and special_cond:
        return True
    elif bear_cond1 and bear_cond2 and bear_cond3 and special_cond:
        return True
    else:
        return False

ohlc_df['engulfing_pattern'] = ohlc_df.apply(find_engulfing_pattern, axis=1)
ohlc_df

,time,open,high,low,close,hammer_pattern,previous_open,previous_high,previous_low,previous_close,engulfing_pattern
0,2023-11-01 04:00:00,1977.079,1977.864,1976.838,1977.767,False,NaN,NaN,NaN,NaN,False
1,2023-11-01 04:05:00,1977.771,1977.827,1977.200,1977.418,False,1977.079,1977.864,1976.838,1977.767,False
2,2023-11-01 04:10:00,1977.440,1977.662,1975.758,1975.814,False,1977.771,1977.827,1977.200,1977.418,False
3,2023-11-01 04:15:00,1975.870,1976.219,1975.231,1975.867,False,1977.440,1977.662,1975.758,1975.814,False
4,2023-11-01 04:20:00,1975.864,1976.382,1975.582,1975.731,False,1975.870,1976.219,1975.231,1975.867,False
...,...,...,...,...,...,...,...,...,...,...,...
751,2023-11-03 20:35:00,1992.453,1992.698,1992.413,1992.504,False,1992.589,1992.650,1992.211,1992.463,False
752,2023-11-03 20:40:00,1992.504,1992.506,1992.307,1992.451,False,1992.453,1992.698,1992.413,1992.504,False
753,2023-11-03 20:45:00,1992.414,1992.417,1992.212,1992.280,False,1992.504,1992.506,1992.307,1992.451,False
754,2023-11-03 20:50:00,1992.291,1992.580,1992.227,1992.471,False,1992.414,1992.417,1992.212,1992.280,False


In [23]:
# visualizing engulfs
engulfs = ohlc_df[ohlc_df['engulfing_pattern'] == True]

fig_engulf = go.Figure(fig)
fig_engulf.update_layout(title='Engulfing Candlestick Patterns')

for i, x in engulfs.iterrows():
    fig_engulf.add_vline(x=x['time'], line_width=2, line_dash="dash", opacity=0.5, line_color="green")
    
fig_engulf